In [1]:
import string
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import string # "string" module is already installed with Python
string.punctuation
from nltk.corpus import stopwords
import re 
import pandas as pd
from nltk.tokenize.treebank import TreebankWordDetokenizer
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from nrclex import NRCLex

In [3]:
# df = pd.read_csv('big_data_cleaned.csv')

In [ ]:
import re

def emoji_strip(text):
    emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text) # no emoji

In [ ]:
def preprocessing(sentence):
#     Lower
    sentence = sentence.lower()

#     punctuation
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '')

#   no numbers
    sentence = ''.join(char for char in sentence if not char.isdigit())

    sentence = emoji_strip(sentence)

    clean_titles = lemmatize(sentence)

    return clean_titles


In [ ]:
def lemmatize(text):
    # Lemmatizing the verbs
    verb_lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "v") # v --> verbs
        for word in text
    ]

# 2 - Lemmatizing the nouns
    noun_lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "n") # n --> nouns
        for word in text
    ]
    return text

In [ ]:
df.title = df.title.apply(preprocessing)

In [ ]:
# !python -m textblob.download_corpora

In [ ]:
sent_dict = {}
basic_sent_dict={}
for i in df.title:
    emotion = NRCLex(i)
    # print(f'TITLE: {i}\n')
    # print(f'affect dictionary: {emotion.affect_dict}\n')
    # print(f'words: {emotion.words}\n')
    # print(f'sentences: {emotion.sentences}\n')
    # print(f'affect list: {emotion.affect_list}\n')
    # print(f'raw emotion score: {emotion.raw_emotion_scores}\n')
    # print(f'top emotions: {emotion.top_emotions}\n')
    # print(f'affect frequency: {emotion.affect_frequencies}\n')
    
    
# advanced sentiment dictionary
    sent_dict[i] = emotion.affect_frequencies
    
# basic sentiment dictionary
    if 'negative' in emotion.raw_emotion_scores:
        basic_sent_dict[i] = 0
    elif 'positive' in emotion.raw_emotion_scores:
        basic_sent_dict[i] = 1
    else:
        basic_sent_dict[i] = 0.5

# merge basic sent df
df_basic_sent = pd.DataFrame.from_dict(basic_sent_dict, orient = "index", columns = ['sentiment']).reset_index()

df_basic_sent.rename(columns = {'index':'title'}, inplace = True)

df = df.merge(df_basic_sent, on='title')

# merge sentiment df
df_sent = pd.DataFrame.from_dict(sent_dict,
                                 orient = "index",
                                 columns = ['anger',
                                            'anticip',
                                            'trust', 
                                            'surprise', 
                                            'positive', 
                                            'negative', 
                                            'sadness', 
                                            'disgust',
                                            'joy']).reset_index()

df_sent.rename(columns = {'index':'title'}, inplace = True)

df = df.merge(df_sent, on='title')

# df

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(round(df.corr(),3), cmap = "coolwarm", annot = True, annot_kws = {"size":10});

In [ ]:
(df['negative'] != 0).sum()

In [ ]:
(df['positive'] != 0).sum()

In [ ]:
negative_ratio = (df['negative'] != 0).sum()/df.shape[0]
print(f'Negative ratio is {negative_ratio*100}%')

In [ ]:
positive_ratio = (df['positive'] != 0).sum()/df.shape[0]
print(f'Positive ratio is {positive_ratio*100}%')

In [ ]:
neutral_ratio = (df.shape[0] - ((df['negative'] != 0).sum() + (df['positive'] != 0).sum()))/df.shape[0]
print(f'Neutral ratio is {neutral_ratio*100}%')

In [ ]:
_ratio = (df['negative'] != 0).sum()/df.shape[0]
print(f'Negative ratio is {negative_ratio*100}%')

In [ ]:
  # declaring data
data = [(df['anger'] != 0).sum(), (df['anticip'] != 0).sum(),
        (df['trust'] != 0).sum(), (df['surprise'] != 0).sum(),
        (df['sadness'] != 0).sum(), (df['disgust'] != 0).sum(),
        (df['joy'] != 0).sum()]

keys = ['Anger', 'Anticipation', 'Trust', 'Surprise', 'Sadness', 'Disgust', 'Joy']
  
# declaring exploding pie
# explode = [0, 0.1, 0, 0, 0]
# define Seaborn color palette to use
palette_color = sns.color_palette('dark')
  
# plotting data on chart
plt.pie(data, labels=keys, colors=palette_color, autopct='%.0f%%')
  
# displaying chart
plt.show()

In [ ]:
!pip install --quiet plotly

In [ ]:
plt.hist(df.sentiment);

In [ ]:
df_high = df[df.views > 200000000]

In [ ]:
sns.regplot(data=df_high, x='sentiment', y='views')

In [ ]:
from scipy.stats import kendalltau, pearsonr, spearmanr

def kendall_pval(x,y):
    return kendalltau(x,y)[1]

def pearsonr_pval(x,y):
    return pearsonr(x,y)[1]

def spearmanr_pval(x,y):
    return spearmanr(x,y)[1]

In [ ]:
df.corr(method='spearman')

In [ ]:
df.corr(method=spearmanr_pval)

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(round(df.corr(),2), cmap = "coolwarm", annot = True, annot_kws = {"size":10});

In [ ]:
sns.scatterplot(x='sentiment', y='views', data=df);

In [ ]:
df_pos_lose = df_0.query('sentiment==1')
df

In [ ]:
df_pos_win = df_10.query('sentiment==1')

In [ ]:
plt.hist(df_pos.views, bins=100);

In [ ]:
df_positive = df_4.query('sentiment==1')
df_positive.mean()

In [ ]:
df_negative = df_6.query('sentiment==-0')
df_negative.mean()

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(round(df_6.corr(),2), cmap = "coolwarm", annot = True, annot_kws = {"size":10});

In [ ]:
df_trim = df[['title', 'views', 'sentiment']]

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(round(df_trim.corr(),2), cmap = "coolwarm", annot = True, annot_kws = {"size":10});

In [ ]:
df_0 = df.query('0<views<100')
df_1 = df.query('0<views<1000') 
df_2 = df.query('1000<views<10_000')
df_3 = df.query('10_000<views<100_000') 
df_4 = df.query('100_000<views< 500_000') 
df_5 = df.query('500_000<views<1_000_000') 
df_6 = df.query('500_000<views<1_000_000') 
df_7 = df.query('1_000_000<views<10_000_000') 
df_8 = df.query('10_000_000<views<50_000_000') 
df_9 = df.query('50_000_000<views<100_000_000') 
df_10 = df.query('100_000_000<views<1000_000_000') 

fig, axes = plt.subplots(4,3, figsize=(15,15))

ax0 = axes[3,1]
ax0.hist(df_0.views, bins=40)
ax0.set_xlabel('views')
ax0.set_ylabel('count')
ax0.set_title('0-100 views')
# plt.xticks(df_0.views, range(0, 100, 10))

ax1 = axes[0,0]
ax1.hist(df_1.views, bins=40)
ax1.set_xlabel('views')
ax1.set_ylabel('count')

ax2 = axes[0,1]
ax2.hist(df_2.views, bins=40)
ax2.set_xlabel('views')
ax2.set_ylabel('count')

ax3 = axes[0,2]
ax3.hist(df_3.views, bins=40)
ax3.set_xlabel('views')
ax3.set_ylabel('count')

ax4 = axes[1,0]
ax4.hist(df_4.views, bins=40)
ax4.set_xlabel('views')
ax4.set_ylabel('count')

ax5 = axes[1,1]
ax5.hist(df_5.views, bins=40)
ax5.set_xlabel('views')
ax5.set_ylabel('count')

ax6 = axes[1,2]
ax6.hist(df_6.views, bins=40)
ax6.set_xlabel('views')
ax6.set_ylabel('count')

ax7 = axes[2,0]
ax7.hist(df_7.views, bins=40)
ax7.set_xlabel('views')
ax7.set_ylabel('count')

ax8 = axes[2,1]
ax8.hist(df_8.views, bins=40)
ax8.set_xlabel('views')
ax8.set_ylabel('count')

ax9 = axes[2,2]
ax9.hist(df_9.views, bins=40)
ax9.set_xlabel('views')
ax9.set_ylabel('count')

ax10 = axes[3,0]
ax10.hist(df_10.views, bins=40)
ax10.set_xlabel('views')
ax10.set_ylabel('count');




In [ ]:
df_1 = df.query('0<views< 1000') 
df_2 = df.query('1000<views<10_000')
df_3 = df.query('10_000<views<100_000') 
df_4 = df.query('100_000<views< 500_000') 
df_5 = df.query('500_000<views<1_000_000') 
df_6 = df.query('500_000<views<1_000_000') 
df_7 = df.query('1_000_000<views<10_000_000') 
df_8 = df.query('10_000_000<views<50_000_000') 
df_9 = df.query('50_000_000<views<100_000_000') 
df_10 = df.query('100_000_000<views<1000_000_000') 
fig, axes = plt.subplots(4,3, figsize=(15,15))
sns.kdeplot(df_1.views, ax=axes[0,0])
sns.kdeplot(df_2.views, ax=axes[0,1])
sns.kdeplot(df_3.views, ax=axes[0,2])
sns.kdeplot(df_4.views, ax=axes[1,0])
sns.kdeplot(df_5.views, ax=axes[1,1])
sns.kdeplot(df_6.views, ax=axes[1,2])
sns.kdeplot(df_7.views, ax=axes[2,0])
sns.kdeplot(df_8.views, ax=axes[2,1])
sns.kdeplot(df_9.views, ax=axes[2,2])
sns.kdeplot(df_10.views, ax=axes[3,0]);

In [ ]:
df_mil = df.query('500_000<views<1_000_000')
plt.hist(df_mil.views, bins=100);
df_mil.shape

In [ ]:
plt.hist(df.views, bins=100);

In [ ]:
basic_sent_dict = {}
for i in df.title:
    emotion = NRCLex(i)
    # print(f'TITLE: {i}\n')
    # print(f'affect dictionary: {emotion.affect_dict}\n')
    # print(f'words: {emotion.words}\n')
    # print(f'sentences: {emotion.sentences}\n')
    # print(f'affect list: {emotion.affect_list}\n')
    # print(f'raw emotion score: {emotion.raw_emotion_scores}\n')
    # print(f'top emotions: {emotion.top_emotions}\n')
    # print(f'affect frequency: {emotion.affect_frequencies}\n')
    # basic_sent_dict[i] = emotion.affect_frequencies
    if 'negative' in emotion.raw_emotion_scores:
        basic_sent_dict[i] = 0
    elif 'positive' in emotion.raw_emotion_scores:
        basic_sent_dict[i] = 1
    else:
        basic_sent_dict[i] = 0.5
# print(basic_sent_dict)
        
    
    
df_basic_sent = pd.DataFrame.from_dict(basic_sent_dict, orient = "index", columns = ['sentiment']).reset_index()

df_basic_sent.rename(columns = {'index':'title'}, inplace = True)

df = df.merge(df_basic_sent, on='title')

df

In [ ]:
def text_cleaner_and_splitter(text):
    return (text.lower()
            .replace('❤️', ' heart ').replace('💯', ' 100 ').replace('❤', ' heart ')
            .replace('🙏', ' pray ').replace('😘', ' kiss ').replace('🤗', ' happy ')
            .replace('💥', ' boom ').replace('✔️', ' like ').replace('😍', ' love ')
            .replace('🐱', ' cat ').replace('💔', ' broken heart ').replace('😵', ' confused ') 
            .replace('😄', ' awesome ').replace('👍', ' thumbs up ').replace('😎', ' cool ')
            .replace('🐷', ' pig ').replace('🤘', " rock'n roll ").replace('🤣', ' laughing hard ')
            .replace('😩', ' oh no ').replace('💎', ' diamond ').replace('😊', ' nice ')
            .replace('☺️', ' very nice ').replace('🙃', ' upside down smile ').replace('🤔', ' not sure ')
            .replace('😂', ' laughing ').replace('🙋🏻‍♀️',' hi ').replace('🥰', ' lovely ').replace('🥺',' sad ')
            .replace('!!!!', '!').replace('!!!', '!').replace('!!', '!').replace('!',' ! ')
            .replace('????', '?').replace('???', '?').replace('??', '?').replace('?',' ? ')
            .replace('oooo','o').replace('oooo','o').replace('ooo','o')
            .replace('..','...').replace('......','...').replace('....','...').replace('....','...').replace('...',' ... ')
            .replace('.',' . ').replace('. . .', ' ... ')
            .replace("'", " ").replace('"', ' ')
            .replace('    ', ' ').replace('   ', ' ').replace('  ', ' ')
            .split())

So priorities are 
- sentiment analysis optimization
- extracting titles
- running analysis with those features extracted

